In [21]:
from Bio import SeqIO
import urllib.parse as up
import os
import psycopg2
import re
from Bio import Entrez
from collections import Counter
from IAP_seqs import Seq

# Lista Cenas Importantes

Erro mt random a povoar references

Ver parte das sequences --> Precisamos de mais espaço mas o SQL limita isso  Data too long for column 'Translacion_seq' at row 1

Cannot add or update a child row: a foreign key constraint fails (`Base Dados IAP`.`Genbank`, CONSTRAINT `Genbank-references` FOREIGN KEY (`ID`) REFERENCES `References` (`ID`))

Ligar Base de Dados

In [49]:
import mysql.connector as SQLC   #ligação Bruno

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "Base Dados IAP",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()


Ficheiro GB 

In [23]:
id_genbank_input = input("ID NCBI")

In [24]:
handle = Entrez.efetch(db='nucleotide',retmode='txt',id=id_genbank_input, rettype = "gb") # dowload do ficheiro
record = SeqIO.read(handle, "gb")

In [25]:
# # file = "/home/sa_bruno/Documentos/GitHub/Projeto_LB/ARL15/ARL15.gb"                
# file = "Teste.gb"
# record = SeqIO.read(file, "genbank") 

# Obter todas as variáveis / Povoar Base de Dados 

# Tabela Locus

In [26]:
# Locus 
ID = record.name
Size = len(record.seq)
Molecular_type = record.annotations["molecule_type"] + " " + record.annotations["topology"]
Genbank_division = record.annotations["data_file_division"] 
Modification_date = record.annotations["date"]

In [27]:
print(ID)
print(Size)
print(Molecular_type)
print(Genbank_division)
print(Modification_date)

NC_001497
8024
DNA circular
VRL
13-AUG-2018


In [28]:
try:
        cur.execute(f"""
                INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                    (ID, Size, Molecular_type, Genbank_division, Modification_date))

except:
        print("Erro Povoação Locus")

Erro Povoação Locus


In [29]:
cur.execute(f"""
                INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                    (ID, Size, Molecular_type, Genbank_division, Modification_date))

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Base Dados IAP`.`Locus`, CONSTRAINT `fk_Locus_1` FOREIGN KEY (`ID`) REFERENCES `Genbank` (`ID_version`))

# Tabela Genbank

Usamos


In [30]:
# Genbank
Organism = record.annotations["organism"]
Accession = record.annotations["accessions"][0]
Definition = record.description
Keywords = ""
ID_version = record.id
for c in record.annotations["keywords"]:
    Keywords += c + ", "
Keywords = Keywords[0: (len(Keywords) -2)]

Sequence = str(record.seq)

In [31]:
print(ID_version)
print(Organism)
print(Accession)
print(Definition)
print(Keywords)
print(Sequence)

NC_001497.2
Cauliflower mosaic virus
NC_001497
Cauliflower mosaic virus, complete genome
RefSeq, genome
TGGTATCAGAGCCATGAATCGGTTTAAGACCAAAACTCAAGAGGGTAAAACCTCACCAAAATACGAAAGAGTTCTTAACTCTAAAAATAAAAGATCTTTCAAGATCAAACATAGTTCCCTCACACCGGTGACCGACAGGATTACCACCGTAAGGTTTCAGAACAACATCGAAAGCGTTTACGCCAACTTCGACTCTCAACTCAAGTCGTCGTACGATGGTAGATCTAAAAAGATCAAGACTCTAAGCCTTAAAAATCTTAGATGTTACGAAGCCTTCCTCAGGAAGTACCTTCTGGAACAATAAATCTCTCTGAGAATAGTACTCTATTGAGTATCCACAGGAAAAATAACCTTCTGTGTTGAGATGGATTTGTATCCAGAAGAAAATACCCAAAGCGAGCAATCGCAGAATTCTGAAAATAATATGCAAATATTTAAATCAGAAAATTCGGATGGATTCTCCTCCGATCTAATGATCTCAAACGATCAATTAAAAAATATCTCTAAAACCCAATTAACCTTGGAGAAAGAAAAGATATTTAAAATGCCTAACGTTTTATCTCAAGTTATGAAAAAAGCGTTTAGCAGGAAAAACGAGATTCTCTACTGCGTCTCGACAAAAGAATTATCAGTGGACATTCACGATGCCACAGGTAAGGTATATCTTCCCTTAATCACTAAGGAAGAGATAAATAAAAGACTTTCCAGCTTAAAACCTGAAGTCAGAAAGACCATGTCCATGGTTCATCTTGGAGCGGTCAAAATATTGCTTAAAGCTCAATTTCGAAATGGGATTGATACCCCAATCAAAATTGCTTTAATCGATGATAGAATCAATTCTAGAAGAGATTGTCTTCTTGGTGCAGCCAAAGGTAATCTAGCATACGGTAAGTTTA

In [32]:
try:
        cur.execute(f"""
                 INSERT INTO Genbank (ID_version, Organism, Accession, Definition, Keywords)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                        (ID_version, Organism, Accession, Definition, Keywords))
except:
        print("Erro Povoação Genbank")

Erro Povoação Genbank


In [33]:
cur.execute(f"""
                 INSERT INTO Genbank (ID_version, Organism, Accession, Definition, Keywords)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                        (ID_version, Organism, Accession, Definition, Keywords))

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Base Dados IAP`.`Genbank`, CONSTRAINT `fk_Genbank_1` FOREIGN KEY (`ID_version`) REFERENCES `genbank_references` (`Id_version_connect`))

# Tabela Sequences  / Parte para Algoritmos

In [34]:
alg_seq = Seq(Sequence).count_nucleotides()

Count_A = alg_seq[0]
Count_C = alg_seq[1]
Count_G = alg_seq[2]
Count_T = alg_seq[3]

In [35]:
print(ID)
print(Sequence)
print(Count_A)
print(Count_C)
print(Count_G)
print(Count_T)

NC_001497
TGGTATCAGAGCCATGAATCGGTTTAAGACCAAAACTCAAGAGGGTAAAACCTCACCAAAATACGAAAGAGTTCTTAACTCTAAAAATAAAAGATCTTTCAAGATCAAACATAGTTCCCTCACACCGGTGACCGACAGGATTACCACCGTAAGGTTTCAGAACAACATCGAAAGCGTTTACGCCAACTTCGACTCTCAACTCAAGTCGTCGTACGATGGTAGATCTAAAAAGATCAAGACTCTAAGCCTTAAAAATCTTAGATGTTACGAAGCCTTCCTCAGGAAGTACCTTCTGGAACAATAAATCTCTCTGAGAATAGTACTCTATTGAGTATCCACAGGAAAAATAACCTTCTGTGTTGAGATGGATTTGTATCCAGAAGAAAATACCCAAAGCGAGCAATCGCAGAATTCTGAAAATAATATGCAAATATTTAAATCAGAAAATTCGGATGGATTCTCCTCCGATCTAATGATCTCAAACGATCAATTAAAAAATATCTCTAAAACCCAATTAACCTTGGAGAAAGAAAAGATATTTAAAATGCCTAACGTTTTATCTCAAGTTATGAAAAAAGCGTTTAGCAGGAAAAACGAGATTCTCTACTGCGTCTCGACAAAAGAATTATCAGTGGACATTCACGATGCCACAGGTAAGGTATATCTTCCCTTAATCACTAAGGAAGAGATAAATAAAAGACTTTCCAGCTTAAAACCTGAAGTCAGAAAGACCATGTCCATGGTTCATCTTGGAGCGGTCAAAATATTGCTTAAAGCTCAATTTCGAAATGGGATTGATACCCCAATCAAAATTGCTTTAATCGATGATAGAATCAATTCTAGAAGAGATTGTCTTCTTGGTGCAGCCAAAGGTAATCTAGCATACGGTAAGTTTATGTTTACTGTATACCCTAAGTTTGGAATAAGCCTTAACACCCAAAGACTTAACCAAACCCTAAGCCTTATTCATGATTTTGAAAATAAAAATCT

In [36]:
try:

        cur.execute("""
                INSERT INTO Sequences (ID, Sequence, Count_A, Count_C, Count_T, Count_G)    
                VALUES (%s, %s, %s, %s, %s,%s)
                """,
                        (ID, Sequence, Count_A, Count_C, Count_T, Count_G))   

except:
        print("Erro Povoação Sequences")

Erro Povoação Sequences


# Tabela Features

In [41]:
temp_l = []
for c in range(len(record.features)):
    temp_l.append(record.features[c].type)

y = Counter(temp_l)
Gene = y["gene"]
# Regulatory = 
#(ID, CDS, Gene, Regulatory, exons, poli_A-site, misc_feature, mRNA))      # acabar esta parte dps  easy
# so contar apartir daqui 
  # Counter({'gene': 5125, 'CDS': 4997, 'tRNA': 105, 'rRNA': 22, 'source': 1, 'tmRNA': 1})
CDS_count = y["CDS"]
Regulatory = y["regulatory"]
exons = y["exons"]
poli_A_site = y["poli_A-site"]                    #  poli_A-site
misc_feature = y["misc_feature"]
mRNA = y["mRNA"]
Location_span = str(record.features[0].location)

In [42]:
print(ID)
print(CDS_count)
print(Gene)
print(Regulatory)
print(poli_A_site)
print(misc_feature)
print(mRNA)
print(Location_span)

NC_001497
7
7
0
0
2
0
[0:8024](+)


In [43]:
try:
        cur.execute("""
        INSERT INTO Features (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 
except:
        print("Erro Features")

Erro Features


In [52]:
cur.execute("""
        INSERT INTO Features (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 

ProgrammingError: 1054 (42S22): Unknown column 'ID' in 'field list'

# Tabela CDS

In [45]:
# usar para dps
CDS_list = []

ID_cds = []
Translacion_seq = []
Localization = []
ID_protein = []
Protein = []
for c in range(len(record.features)):
    if record.features[c].type == "CDS":
        CDS_list.append(record.features[c])

# Translacion_seq
for c in range(len(CDS_list)):
    ID_cds.append(CDS_list[c])
    Translacion_seq.append(CDS_list[c].qualifiers["translation"])
    Localization.append(CDS_list[c].location)
    ID_protein.append(CDS_list[c].qualifiers["protein_id"])
    try:
        Protein.append(CDS_list[c].qualifiers["product"])
    except:
        Protein.append("------")
    


In [46]:
for c in range(len(ID_cds)):
    print(ID_cds[c])
    print()
    print(Translacion_seq[c])
    print()
    print(Localization[c])
    print()
    print(ID_protein[c])
    print()
    print(Protein[c])
    print()
    print()


type: CDS
location: [<1:304](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GeneID:1489547']
    Key: locus_tag, Value: ['CaMVgp1']
    Key: note, Value: ['Predicted by GeneMark']
    Key: product, Value: ['hypothetical protein']
    Key: protein_id, Value: ['NP_597788.1']
    Key: translation, Value: ['GIRAMNRFKTKTQEGKTSPKYERVLNSKNKRSFKIKHSSLTPVTDRITTVRFQNNIESVYANFDSQLKSSYDGRSKKIKTLSLKNLRCYEAFLRKYLLEQ']


['GIRAMNRFKTKTQEGKTSPKYERVLNSKNKRSFKIKHSSLTPVTDRITTVRFQNNIESVYANFDSQLKSSYDGRSKKIKTLSLKNLRCYEAFLRKYLLEQ']

[<1:304](+)

['NP_597788.1']

['hypothetical protein']


type: CDS
location: [364:1348](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GOA:P03545', 'InterPro:IPR001022', 'UniProtKB/Swiss-Prot:P03545', 'GeneID:1489544']
    Key: locus_tag, Value: ['CaMVgp2']
    Key: note, Value: ['reading frame (I)']
    Key: product, Value: ['Movement protein']
    Key: protein_id, Value: ['NP_056724.1']
    Key: translation, Value: ['M

In [51]:
num_list = 0                            # cdc ID que falta ver
for c in range(len(ID_cds)):
        cur.execute("""
                INSERT INTO CDS (ID_protein_cds, Translacion_seq, Localization, Protein)    
                VALUES (%s, %s, %s, %s)
                """,
                (str(ID_protein[num_list]), str(Translacion_seq[num_list]), str(Localization[num_list]), str(Protein[num_list])))
        num_list += 1

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`Base Dados IAP`.`CDS`, CONSTRAINT `cds- feature` FOREIGN KEY (`ID_protein_cds`) REFERENCES `Features` (`ID_source`))

# Tabela References

In [ ]:
def get_authors(a):
    exis_5 = re.search(r"authors:\s+([\w\s,\.-]+)", a)          #dúvida (como é que o ciclo sabe se isto está vazio ou não)       
    if exis_5:
        Authors = []
        Authors_work = []
        y = re.search(r"authors:\s+([\w\s,\.-]+)", a)    # isto aqui é o remark
        Authors_work.append(y.group(1))
        # else:
        #     Consortium.append("------")
        help_authors = [] 
        Authors = str(Authors_work[0]).split("., ")
        for i, c in enumerate(Authors):
            f = c.split(" and ")
            help_authors.append([i, f])
        c = len(Authors)
        i = 0
        while c > 0:
            if len(help_authors[i][1]) > 1:
                Authors.remove(Authors[i])
                for o in help_authors[i][1]:
                    Authors.append(o)

            c -= 1
            i += 1
    else:
        Authors = []

    return Authors



In [ ]:
# referencias  # faz parte do PubMed INfo
Pubmed = []
Title = []
Journal = []
Consortium = []
Remark = []
Authors = []
b = ""
pubmed_trash = record.annotations["references"]    # resolver isto 
c = len(pubmed_trash)
i = 0
while c > 0:
    a = str(pubmed_trash[i]).replace("\n", "§")   
    
    exist_3 = re.search(r"pubmed id:\s+(\d+)", a)
    if exist_3:
        m = re.search(r"pubmed id:\s+(\d+)", a)                 # COnfirmar PubMEd haver sempre/nao
        Pubmed.append(m.group(1))
    else:
        Pubmed.append("None")

    
    n = re.search(r"title:\s+([\w\s')(-]+)", a)
    Title.append(n.group(1))

    exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)
    if exist_2:
        j = re.search(r"journal:\s+([\w\s.()--,]+)", a)
        Journal.append(j.group(1))
    else:
        Journal.append("None")

    exis = re.search(r"consrtm:\s+([\w\s]+)", a)
    if exis:
        k = re.search(r"consrtm:\s+([\w\s]+)", a)    # isto aqui é o remark
        Consortium.append(k.group(1))
    else:
        Consortium.append("None")         # ter cuidado com isto dps
    

    exis_1 = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\s\d])", a)
    if exis_1:
        l = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\d\s:\[;\(\)\.,!?<>{}\"]+)", a)
        Remark.append(l.group(1))
    else:
        Remark.append("None") 

    Authors.append(get_authors(a))


    
    c -= 1
    i += 1

/tmp/ipykernel_3961/713990561.py:26: FutureWarning: Possible set difference at position 20
  exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)


In [ ]:
print(ID)
print(Pubmed)
print(Authors)
print(Title)
print(Journal)
print(Consortium)
print(Remark)

NC_003539
['1754400', 'None', 'None', 'None']
[['Puchta,H', 'Ramm,K', 'Luckinger,R', 'Hadas,R', 'Bar-Joseph,M.', 'Snger,H.L.'], ['Puchta,H', 'Ramm,K', 'Luckinger,R', 'Freimueller,K.', 'Saenger,H.L.'], [], ['Saenger,H.L.']]
['Primary and secondary structure of citrus viroid IV (CVd IV)', 'Citrus Viroid IV (CVd IV)', 'Direct Submission', 'Direct Submission']
['Nucleic Acids Res. 19 (23), 6640 (1991)', 'Unpublished', 'Submitted (09-MAY-2007) National Center for Biotechnology Information, NIH, Bethesda, MD 20894, USA', 'Submitted (09-MAR-1989) Saenger H.L., Max-Planck-Institut fuer Biochemie, Am Klopferspitz, D-8033 Martinsried bei Muenchen, FRG']
['None', 'None', 'NCBI Genome Project', 'None']
['None', 'None', 'None', 'None']


In [ ]:
reference_count = 0                  # erro weird 
for c in Pubmed:    
        cur.execute("""
                INSERT INTO references (ID, Pubmed, ID_authors, Title, Journal, Consortium, Remark)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """,
                (ID + f"_{reference_count}", str(Pubmed[reference_count]), str(Authors[reference_count]), str(Title[reference_count]), str(Journal[reference_count]), str(Consortium[reference_count]), str(Remark[reference_count])))
        reference_count += 1

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'references (ID, Pubmed, ID_authors, Title, Journal, Consortium, Remark)
        ' at line 1

# Tabela Authors


In [ ]:
for c in Authors:
        for i, a in enumerate(c):
                cur.execute("""
        INSERT INTO Authors (ID_authors, Names)    
        VALUES (%s, %s)
        """,
                (str(c) + f"_{i}", str(a)))

ProgrammingError: 1054 (42S22): Unknown column 'ID_authors' in 'field list'

# Tabela Pubmed_Info

In [ ]:
Abstract = []                        # mudar para cada referenias 
DOI = []
Affiliation = []
for c in Pubmed:
    if c != "None":
        Entrez.email = 'sa.bruno.2001@gmail.com'
        # handle = Entrez.efetch(db='pubmed',retmode='xml',id="15496913")
        handle = Entrez.efetch(db='pubmed',retmode='xml',id=c)
        resultado = Entrez.read(handle)
        q = str(resultado["PubmedArticle"][0])
        
        if re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q):
            o = re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q)
            Abstract.append(o.group(1))
        else:
            Abstract.append("None")
        
        if re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q):
            p = re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q)
            DOI.append(p.group(1))
        else:
            DOI.append("None")

        if re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q):
            t = re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q)
            Affiliation.append(t.group(1))
        else:
            Affiliation.append("None")
    if c == "None":
            Abstract.append("None")
            DOI.append("None")
            Affiliation.append("None")



In [ ]:
print(Pubmed)
print(Title)
print(Abstract)
print(DOI)


['1754400', 'None', 'None', 'None']
['Primary and secondary structure of citrus viroid IV (CVd IV)', 'Citrus Viroid IV (CVd IV)', 'Direct Submission', 'Direct Submission']
['None', 'None', 'None', 'None']
['PMC329237', 'None', 'None', 'None']


In [ ]:
# try:
#         pubmed_count= 0
#         for c in Pubmed:
#                 cur.execute("""
#                         INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, DOI)   
#                         VALUES (%s, %s, %s, %s)
#                         """,
#                         (Pubmed[pubmed_count] + f"_{pubmed_count}", Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count]))   
#         if c != "None":
#                 pubmed_count += 1
# except:
#         print("Erro Povoar Pubmed")

In [ ]:
pubmed_count= 0
for c in Pubmed :
        if c != "None":
                cur.execute("""
                        INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, DOI, Affiliation)   
                        VALUES (%s, %s, %s, %s, %s)
                        """,
                        (Pubmed[pubmed_count], Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count], Affiliation[pubmed_count]))  
                pubmed_count += 1 
        else:
                pubmed_count += 1

# Lixo para baixo

In [ ]:
#     cur.execute("""
#         INSERT INTO Authors (ID_authors, Affiliation)    
#         VALUES (%s, %s, %s))
#         """,
#         (Authors, Affiliation))    